In [1]:
#! pip install ultralytics 
#! pip install psycopg2
import numpy as np
#import cv2
#import matplotlib.pyplot as plt
import os
import psycopg2
#import im_utils
import db_ops
import pandas as pd

import cv2

import tensorflow as tf
print(tf.__version__)

from keras.models import Model
from keras.saving import load_model

Connecting to the PostgreSQL database...
Connection successful


2024-07-24 12:52:45.440319: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 12:52:45.700641: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-24 12:52:45.702653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 12:52:46.989658: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0


In [1]:

def bb_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [4]:
test_lbls = ['5-hour energy', '99', 'AW', 'Acadia', 'Arizona Tea', 'Bang', 'Big Gulp', 'Bill Miller', 'Boulevard Brewing', 'Brisk', 'Bud', 'Budweiser', 'Burger King', 'Busch', 'Camel', 'Cheetos', 'Coca Cola', 'Coors', 'Corona', 'Dasani', 'Dasy', 'David', 'Deer Park', 'Dorritos', 'Dunkin Donuts', 'Evan Williams', 'Fanta', 'Fireball', 'Flow', 'Fruit Punch', 'Gatorade', 'GoGo', 'Google', 'Goya', 'Grace', 'Green Mountain', 'Heineken', 'Hersheys', 'Hill Country Fair', 'Icehouse Edge', 'Jarritos', 'Jim Beam', 'Kirkland', 'Kit Kat', 'Kool-Aid', 'La Yogurt', 'LaCroix', 'Lays', 'Lybbis', 'Marlboro', 'McDonalds', 'Mentos', 'Michelob', 'Miller', 'Milwakee Best Ice', 'MnM', 'Modelo', 'Monster', 'Mountain Dew', 'Natural Ice', 'New Amsterdam', 'Niagara', 'Oreo', 'Pabst Blue Ribbon', 'Pacifico', 'Pampers', 'Parliament', 'Peaceful Fruits', 'Perrier', 'Poland Spring', 'Popeyes', 'PurAqua', 'Redbull', 'Ross Ouzo', 'Sams Club', 'Samuel Adams', 'Sangria Senorial', 'Sanpple', 'Seven eleven', 'Shasta', 'Silver Falls', 'Slurpee', 'Smirnoff', 'Snickers', 'Sprite', 'Stella Artois', 'Turkey Hill', 'Twix', 'Two Ball', 'Vitamin', 'Welchs', 'Yoohoo']

In [6]:
label_path = '../random_test_dataset/train/labels'
files = os.listdir(label_path)
file_names = []
labels = []
lstX1 = []
lstY1 = []
lstX2 = []
lstY2 = []

for file in files:
    file_name = file.split('_jpg')[0]
    with open(label_path + '/'+file, 'r') as label_file:
        for line in label_file:
            idx, x1, y1, x2, y2 = line.split(' ')
            lstX1.append(float(x1))
            lstX2.append(float(x2))
            lstY1.append(float(y1))
            lstY2.append(float(y2))
            labels.append(test_lbls[int(idx)].lower())
            file_names.append(file_name)
        
test_labels_df = pd.DataFrame({'file_name':file_names,
                               'label': labels,
                                'x1': lstX1,
                                'x2': lstX2,
                                'y1': lstY1,
                                'y2': lstY2,}, index = range(len(file_names)))
test_labels_df

,file_name,label,x1,x2,y1,y2
0,IMG_1189,kirkland,0.453869,0.124008,0.369544,0.076885
1,IMG_2834,bud,0.496858,0.124008,0.294519,0.135169
2,IMG_2834,bud,0.748181,0.170304,0.608259,0.125248
3,IMG_4260,kirkland,0.559276,0.054563,0.426587,0.128968
4,IMG_8921,pabst blue ribbon,0.447715,0.073853,0.504162,0.050429
...,...,...,...,...,...,...
305,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.568321,0.050040,0.788399,0.068084
306,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.560150,0.047999,0.875543,0.068084
307,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,heineken,0.536664,0.035744,0.958607,0.081699
308,gMyVYZ2G2PM9Z8QlQc4Wt2QmwJI1bXHIZd6zEIIO,gatorade,0.699550,0.069445,0.296842,0.092594


In [ ]:
# load model
model = load_model("./B0_custaug_model/B0_custaug_model.keras")

# load dataframe that correlated model index and database ID
id_idx_df = pd.read_csv('brand_ids_idx.csv', sep = '\t')